In [1]:
teps='''WNK3
AASS
PARP14
MLLT1
HAO1
HCN4
NUDT7
RIPK2
FAM83A
GALT
CDK12
PHIP
HDAC6
RECQL5
JMJD1B
JMJD2D
SETDB1
PfBDP4B
LIMK1
DPAGT1
DCLRE1A
TMEM16K
TASK1
SLC1A1
STAG1
TBXT
KLHL20
KEAP1
KALRN
ALAS2
MTHFR
RAC1
FAM83B
GALK
MLLT3'''.split()

In [2]:
import csv
with open('../GEL-PanelApp-analysis/tally.csv','r') as fh:
    tally = {t['uniprot']: t for t in csv.DictReader(fh)}

In [3]:
import json
namedex = json.load(open('data/human_prot_namedex.json','r'))

In [4]:
with open('teps_panelApp_old.csv','w',newline='\n') as fh:
    w = csv.DictWriter(fh,fieldnames=['TEP_name', 'status']+list(tally['Q6ZN28'].keys()))
    w.writeheader()
    for p in teps:
        if p in namedex:
            if namedex[p] in tally:
                data = tally[namedex[p]]
                data['status'] = '✓ (in PanelApp)'
                data['TEP_name'] = p
            else:
                data = {'TEP_name': p, 'status': '✗ (not in PanelApp)'}
        else:
            data = {'TEP_name': p, 'status': '‼ (not a valid gene name)'}
        w.writerow(data)
print('Done.')

NameError: name 'namedex' is not defined

In [4]:
import json, csv

from warnings import warn

from collections import Counter
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import os

from multiprocessing.pool import ThreadPool as Pool
from threading import Lock

from pprint import PrettyPrinter
pprint = PrettyPrinter().pprint

from protein.generate import ProteinGatherer
Protein = ProteinGatherer

In [5]:
#name	full_name	tally	panel	ensembl	location	uniprot	tags	OMIM	panels	
#panel_names	disease_group	disease_sub_group	mode_of_inheritance	confidence_level	len	
#pLI	pRec	pNull	%modelled
#name	full_name	tally	panel	ensembl

keys = ('uniprot', 'pLI', 'pRec', 'pNull','ensembl')
def get_data(name):
    data = {}
    data['name'] = name
    protein = Protein(gene_name=name)
    if name in namedex:
        protein.uniprot = namedex[name]
        data['uniprot'] = protein.uniprot
    else:
        protein.uniprot = 'ERROR'
        return data
    try:
        protein.parse_uniprot()
        protein.parse_pLI()
        protein.parse_swissmodel()
    except ValueError as err:
        print(str(err))
    data = {'%modelled': protein.get_percent_modelled(),
            'name': protein.gene_name,
            'ensembl': protein.ENSG,
            'full_name': protein.recommended_name,
            **{k: getattr(protein, k) for k in keys}}
    return data

with open('teps_panelApp.csv','w',newline='\n') as fh:
    w = csv.DictWriter(fh,fieldnames=['TEP_name', 'status']+list(tally['Q6ZN28'].keys()))
    w.writeheader()
    n=0
    for p in teps:
        n+=1
        if p in namedex:
            print(f'{p} ({n}/{len(teps)})')
            if namedex[p] in tally:
                data = tally[namedex[p]]
                data['status'] = '✓ (in PanelApp)'
                data['TEP_name'] = p
            else:
                print('needs parsing')
                data = {'TEP_name': p, 'status': '✗ (not in PanelApp)', **get_data(p)}
                
        else:
            data = {'TEP_name': p, 'status': '‼ (not a valid gene name)'}
        w.writerow(data)
print('Done.')

WNK3 (1/35)
AASS (2/35)
PARP14 (3/35)
needs parsing


/home/matteo/Coding/protein-module-for-VENUS/protein/generate/_protein_gatherer.py:585: UserWarning: error arose in %modelled...'Structure' object is not subscriptable
  warn(f'error arose in %modelled...{err}')
/home/matteo/Coding/protein-module-for-VENUS/protein/generate/_protein_base_mixin.py:36: UserWarning: Accessed non-existant attribute ensembl for Protein instance. Likely cause the code changed, but the from_pickle flag is True.
  warn('Accessed non-existant attribute {item} for Protein instance. Likely cause the code changed, but the from_pickle flag is True.'.format(v=self, item=item))


MLLT1 (4/35)
needs parsing
HAO1 (5/35)
needs parsing
HCN4 (6/35)
NUDT7 (7/35)
needs parsing
RIPK2 (8/35)
needs parsing
FAM83A (9/35)
needs parsing
GALT (10/35)
CDK12 (11/35)
needs parsing
PHIP (12/35)
HDAC6 (13/35)
RECQL5 (14/35)
needs parsing
JMJD1B (15/35)
JMJD2D (16/35)
needs parsing
SETDB1 (17/35)
needs parsing
LIMK1 (19/35)
DPAGT1 (20/35)
DCLRE1A (21/35)
needs parsing
TMEM16K (22/35)
TASK1 (23/35)
SLC1A1 (24/35)
needs parsing
STAG1 (25/35)
TBXT (26/35)
KLHL20 (27/35)
needs parsing
KEAP1 (28/35)
needs parsing
KALRN (29/35)
needs parsing
ALAS2 (30/35)
MTHFR (31/35)
RAC1 (32/35)
FAM83B (33/35)
needs parsing
GALK (34/35)
MLLT3 (35/35)
needs parsing
Done.
